**CSI5139 Assignment 1**

In this assignment we use the CMU Image dataset, and train binary and multiclass linear classifier.

### Setup

In [72]:
import pgm2pil
import PIL.Image
import numpy as np
import os
from scipy import misc
from pprint import pprint
import imageio

origImageOpen = PIL.Image.open
PIL.Image.open = pgm2pil.imageOpenWrapper

### CMU Image dataset

In [73]:
folder = 'datasets/faces/'
folders = [x[0] for x in os.walk(folder)]
folders = folders[1:]
#print(folders)
#print(len(folders))
A1, A2, A4 = np.array([]), np.array([]), np.array([])
Y1, Y2, Y4 = np.array([]), np.array([]), np.array([])
for fld in folders:
    for file in os.listdir(fld):
        labels = file
        file = os.path.join(fld, file)
        
        if os.path.isfile(file) and file.endswith('.pgm'):
        # handle .bad files
        #if file.endswith('.pgm'):

            image = PIL.Image.open(file)
            #image = origImageOpen(file)
            image = np.array(image)
            
            # get target person, head pose, facial expression, sunglasses state
            labels = labels.split('.')
            labels = labels[0].split('_')

            # sort resolution
            print(labels)
            if len(labels) == 4:
                np.append(A1, image)
                np.append(Y1, labels)
            elif len(labels) == 5:
                res = labels[-1]
                labels = labels[:-1]
                if res == '2':
                    np.append(A2, labels)
                    np.append(Y2, labels)
                elif res == '4':
                    np.append(A4, labels)
                    np.append(Y4, labels)       

#print(A1)
#pprint(Y1)

['an2i', 'left', 'angry', 'open']
Shit
['an2i', 'left', 'angry', 'open', '2']
Shit
['an2i', 'left', 'angry', 'open', '4']
['an2i', 'left', 'angry', 'sunglasses']
Shit
['an2i', 'left', 'angry', 'sunglasses', '2']
Shit
['an2i', 'left', 'angry', 'sunglasses', '4']
['an2i', 'left', 'happy', 'open']
Shit
['an2i', 'left', 'happy', 'open', '2']
Shit
['an2i', 'left', 'happy', 'open', '4']
['an2i', 'left', 'happy', 'sunglasses']
Shit
['an2i', 'left', 'happy', 'sunglasses', '2']
Shit
['an2i', 'left', 'happy', 'sunglasses', '4']
['an2i', 'left', 'neutral', 'open']
Shit
['an2i', 'left', 'neutral', 'open', '2']
Shit
['an2i', 'left', 'neutral', 'open', '4']
['an2i', 'left', 'neutral', 'sunglasses']
Shit
['an2i', 'left', 'neutral', 'sunglasses', '2']
Shit
['an2i', 'left', 'neutral', 'sunglasses', '4']
['an2i', 'left', 'sad', 'open']
Shit
['an2i', 'left', 'sad', 'open', '2']
Shit
['an2i', 'left', 'sad', 'open', '4']
['an2i', 'left', 'sad', 'sunglasses']
Shit
['an2i', 'left', 'sad', 'sunglasses', '2']


Shit
['boland', 'right', 'happy', 'sunglasses', '2']
Shit
['boland', 'right', 'happy', 'sunglasses', '4']
['boland', 'right', 'neutral', 'open']
Shit
['boland', 'right', 'neutral', 'open', '2']
Shit
['boland', 'right', 'neutral', 'open', '4']
['boland', 'right', 'neutral', 'sunglasses']
Shit
['boland', 'right', 'neutral', 'sunglasses', '2']
Shit
['boland', 'right', 'neutral', 'sunglasses', '4']
['boland', 'right', 'sad', 'open']
Shit
['boland', 'right', 'sad', 'open', '2']
Shit
['boland', 'right', 'sad', 'open', '4']
['boland', 'right', 'sad', 'sunglasses']
Shit
['boland', 'right', 'sad', 'sunglasses', '2']
Shit
['boland', 'right', 'sad', 'sunglasses', '4']
['boland', 'straight', 'angry', 'open']
Shit
['boland', 'straight', 'angry', 'open', '2']
Shit
['boland', 'straight', 'angry', 'open', '4']
['boland', 'straight', 'angry', 'sunglasses']
Shit
['boland', 'straight', 'angry', 'sunglasses', '2']
Shit
['boland', 'straight', 'angry', 'sunglasses', '4']
['boland', 'straight', 'happy', 'ope

['ch4f', 'up', 'angry', 'open', '4']
['ch4f', 'up', 'angry', 'sunglasses']
['ch4f', 'up', 'angry', 'sunglasses', '2']
['ch4f', 'up', 'angry', 'sunglasses', '4']
['ch4f', 'up', 'happy', 'open']
['ch4f', 'up', 'happy', 'open', '2']
['ch4f', 'up', 'happy', 'open', '4']
['ch4f', 'up', 'happy', 'sunglasses']
['ch4f', 'up', 'happy', 'sunglasses', '2']
['ch4f', 'up', 'happy', 'sunglasses', '4']
['ch4f', 'up', 'neutral', 'open']
['ch4f', 'up', 'neutral', 'open', '2']
['ch4f', 'up', 'neutral', 'open', '4']
['ch4f', 'up', 'neutral', 'sunglasses']
['ch4f', 'up', 'neutral', 'sunglasses', '2']
['ch4f', 'up', 'neutral', 'sunglasses', '4']
['ch4f', 'up', 'sad', 'open']
['ch4f', 'up', 'sad', 'open', '2']
['ch4f', 'up', 'sad', 'open', '4']
['ch4f', 'up', 'sad', 'sunglasses']
['ch4f', 'up', 'sad', 'sunglasses', '2']
['ch4f', 'up', 'sad', 'sunglasses', '4']
['cheyer', 'left', 'angry', 'open']
Shit
['cheyer', 'left', 'angry', 'open', '2']
Shit
['cheyer', 'left', 'angry', 'open', '4']
['cheyer', 'left', 'a

Shit
['danieln', 'left', 'happy', 'sunglasses', '2']
Shit
['danieln', 'left', 'happy', 'sunglasses', '4']
['danieln', 'left', 'neutral', 'open']
Shit
['danieln', 'left', 'neutral', 'open', '2']
['danieln', 'left', 'neutral', 'open', '4']
['danieln', 'left', 'neutral', 'sunglasses']
Shit
['danieln', 'left', 'neutral', 'sunglasses', '2']
Shit
['danieln', 'left', 'neutral', 'sunglasses', '4']
['danieln', 'left', 'sad', 'open']
['danieln', 'left', 'sad', 'open', '2']
['danieln', 'left', 'sad', 'open', '4']
['danieln', 'left', 'sad', 'sunglasses']
Shit
['danieln', 'left', 'sad', 'sunglasses', '2']
Shit
['danieln', 'left', 'sad', 'sunglasses', '4']
['danieln', 'right', 'angry', 'open']
Shit
['danieln', 'right', 'angry', 'open', '2']
Shit
['danieln', 'right', 'angry', 'open', '4']
['danieln', 'right', 'angry', 'sunglasses']
Shit
['danieln', 'right', 'angry', 'sunglasses', '2']
Shit
['danieln', 'right', 'angry', 'sunglasses', '4']
['danieln', 'right', 'happy', 'open']
Shit
['danieln', 'right',

Shit
['karyadi', 'left', 'neutral', 'open', '2']
Shit
['karyadi', 'left', 'neutral', 'open', '4']
['karyadi', 'left', 'neutral', 'sunglasses']
Shit
['karyadi', 'left', 'neutral', 'sunglasses', '2']
Shit
['karyadi', 'left', 'neutral', 'sunglasses', '4']
['karyadi', 'left', 'sad', 'open']
Shit
['karyadi', 'left', 'sad', 'open', '2']
Shit
['karyadi', 'left', 'sad', 'open', '4']
['karyadi', 'left', 'sad', 'sunglasses']
Shit
['karyadi', 'left', 'sad', 'sunglasses', '2']
Shit
['karyadi', 'left', 'sad', 'sunglasses', '4']
['karyadi', 'right', 'angry', 'open']
Shit
['karyadi', 'right', 'angry', 'open', '2']
Shit
['karyadi', 'right', 'angry', 'open', '4']
['karyadi', 'right', 'angry', 'sunglasses']
Shit
['karyadi', 'right', 'angry', 'sunglasses', '2']
Shit
['karyadi', 'right', 'angry', 'sunglasses', '4']
['karyadi', 'right', 'happy', 'open']
Shit
['karyadi', 'right', 'happy', 'open', '2']
Shit
['karyadi', 'right', 'happy', 'open', '4']
['karyadi', 'right', 'neutral', 'open']
Shit
['karyadi', 'r

Shit
['kk49', 'right', 'sad', 'sunglasses', '2']
Shit
['kk49', 'right', 'sad', 'sunglasses', '4']
['kk49', 'straight', 'angry', 'open']
Shit
['kk49', 'straight', 'angry', 'open', '2']
Shit
['kk49', 'straight', 'angry', 'open', '4']
['kk49', 'straight', 'angry', 'sunglasses']
Shit
['kk49', 'straight', 'angry', 'sunglasses', '2']
Shit
['kk49', 'straight', 'angry', 'sunglasses', '4']
['kk49', 'straight', 'happy', 'open']
Shit
['kk49', 'straight', 'happy', 'open', '2']
Shit
['kk49', 'straight', 'happy', 'open', '4']
['kk49', 'straight', 'happy', 'sunglasses']
Shit
['kk49', 'straight', 'happy', 'sunglasses', '2']
Shit
['kk49', 'straight', 'happy', 'sunglasses', '4']
['kk49', 'straight', 'neutral', 'open']
Shit
['kk49', 'straight', 'neutral', 'open', '2']
Shit
['kk49', 'straight', 'neutral', 'open', '4']
['kk49', 'straight', 'neutral', 'sunglasses']
Shit
['kk49', 'straight', 'neutral', 'sunglasses', '2']
Shit
['kk49', 'straight', 'neutral', 'sunglasses', '4']
['kk49', 'straight', 'sad', 'ope

Shit
['mitchell', 'up', 'sad', 'sunglasses', '2']
Shit
['mitchell', 'up', 'sad', 'sunglasses', '4']
['night', 'left', 'angry', 'open']
Shit
['night', 'left', 'angry', 'open', '2']
Shit
['night', 'left', 'angry', 'open', '4']
['night', 'left', 'angry', 'sunglasses']
Shit
['night', 'left', 'angry', 'sunglasses', '2']
Shit
['night', 'left', 'angry', 'sunglasses', '4']
['night', 'left', 'happy', 'open']
Shit
['night', 'left', 'happy', 'open', '2']
Shit
['night', 'left', 'happy', 'open', '4']
['night', 'left', 'happy', 'sunglasses']
Shit
['night', 'left', 'happy', 'sunglasses', '2']
Shit
['night', 'left', 'happy', 'sunglasses', '4']
['night', 'left', 'neutral', 'open']
Shit
['night', 'left', 'neutral', 'open', '2']
['night', 'left', 'neutral', 'open', '4']
['night', 'left', 'neutral', 'sunglasses']
Shit
['night', 'left', 'neutral', 'sunglasses', '2']
Shit
['night', 'left', 'neutral', 'sunglasses', '4']
['night', 'left', 'sad', 'open']
Shit
['night', 'left', 'sad', 'open', '2']
Shit
['night'

Shit
['saavik', 'left', 'sad', 'sunglasses', '2']
Shit
['saavik', 'left', 'sad', 'sunglasses', '4']
['saavik', 'right', 'angry', 'open']
Shit
['saavik', 'right', 'angry', 'open', '2']
Shit
['saavik', 'right', 'angry', 'open', '4']
['saavik', 'right', 'angry', 'sunglasses']
Shit
['saavik', 'right', 'angry', 'sunglasses', '2']
Shit
['saavik', 'right', 'angry', 'sunglasses', '4']
['saavik', 'right', 'happy', 'open']
Shit
['saavik', 'right', 'happy', 'open', '2']
Shit
['saavik', 'right', 'happy', 'open', '4']
['saavik', 'right', 'happy', 'sunglasses']
Shit
['saavik', 'right', 'happy', 'sunglasses', '2']
Shit
['saavik', 'right', 'happy', 'sunglasses', '4']
['saavik', 'right', 'neutral', 'open']
Shit
['saavik', 'right', 'neutral', 'open', '2']
Shit
['saavik', 'right', 'neutral', 'open', '4']
['saavik', 'right', 'neutral', 'sunglasses']
Shit
['saavik', 'right', 'neutral', 'sunglasses', '2']
Shit
['saavik', 'right', 'neutral', 'sunglasses', '4']
['saavik', 'right', 'sad', 'open']
Shit
['saavik

Shit
['sz24', 'straight', 'angry', 'open', '2']
Shit
['sz24', 'straight', 'angry', 'open', '4']
['sz24', 'straight', 'angry', 'sunglasses']
Shit
['sz24', 'straight', 'angry', 'sunglasses', '2']
Shit
['sz24', 'straight', 'angry', 'sunglasses', '4']
['sz24', 'straight', 'happy', 'open']
Shit
['sz24', 'straight', 'happy', 'open', '2']
Shit
['sz24', 'straight', 'happy', 'open', '4']
['sz24', 'straight', 'happy', 'sunglasses']
Shit
['sz24', 'straight', 'happy', 'sunglasses', '2']
Shit
['sz24', 'straight', 'happy', 'sunglasses', '4']
['sz24', 'straight', 'neutral', 'open']
Shit
['sz24', 'straight', 'neutral', 'open', '2']
Shit
['sz24', 'straight', 'neutral', 'open', '4']
['sz24', 'straight', 'neutral', 'sunglasses']
Shit
['sz24', 'straight', 'neutral', 'sunglasses', '2']
Shit
['sz24', 'straight', 'neutral', 'sunglasses', '4']
['sz24', 'straight', 'sad', 'open']
Shit
['sz24', 'straight', 'sad', 'open', '2']
Shit
['sz24', 'straight', 'sad', 'open', '4']
['sz24', 'straight', 'sad', 'sunglasses'